In [3]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df = pd.read_csv('data.csv', header=None, names=['x1','x2','y'])
X, y = df[['x1','x2']].values, df['y'].values

def train_perceptron(X, y, lr=0.2, epochs=65):
    w = np.random.randn(X.shape[1])
    b = 0.0
    history = [(w.copy(), b)]
    for _ in range(epochs):
        for xi, yi in zip(X, y):
            z = np.dot(w, xi) + b
            pred = 1 if z > 0 else 0
            error = yi - pred
            w += lr * error * xi
            b += lr * error
        history.append((w.copy(), b))
    return history

def plot_all_boundaries(X, y, lrs, epochs):
    fig = make_subplots(rows=1, cols=3, subplot_titles=[f"lr={lr}" for lr in lrs])

    for col, lr in enumerate(lrs, start=1):
        hist = train_perceptron(X, y, lr=lr, epochs=epochs)

        for lbl, color in zip([0,1], ['blue','orange']):
            mask = (y == lbl)
            fig.add_trace(go.Scatter(
                x=X[mask, 0], y=X[mask, 1],
                mode='markers', marker=dict(size=6, color=color),
                name=f'Class {lbl}' if col == 1 else None,
                showlegend=(col == 1)
            ), row=1, col=col)

        for i, (w, b) in enumerate(hist):
            x0, x1 = 0.0, 1.0
            if abs(w[1]) > 1e-6:
                y0 = -(w[0] * x0 + b) / w[1]
                y1 = -(w[0] * x1 + b) / w[1]
            else:
                x0 = x1 = -b / w[0]
                y0, y1 = 0.0, 1.0

            if i == 0:
                colr, dash, width, name = 'red', 'dash', 3, 'Initial'
            elif i == len(hist) - 1:
                colr, dash, width, name = 'black', 'solid', 3, 'Final'
            else:
                colr, dash, width, name = 'green', 'dash', 2, None

            fig.add_trace(go.Scatter(
                x=[x0, x1], y=[y0, y1],
                mode='lines',
                line=dict(color=colr, dash=dash, width=width),
                name=name if (col == 1 and name) else None,
                showlegend=(col == 1 and bool(name))
            ), row=1, col=col)

    fig.update_layout(
        title_text=f"Part 1: Heuristic Perceptron (Epochs={epochs})",
        height=500,
        width=1400,
        showlegend=True
    )

    for i in range(1, 4):
        fig.update_xaxes(range=[0, 1], title_text="x₁", row=1, col=i)
        fig.update_yaxes(range=[0, 1], title_text="x₂", row=1, col=i)

    fig.show()

plot_all_boundaries(X, y, lrs=[0.02,0.2, 2], epochs=35)
plot_all_boundaries(X, y, lrs=[0.02,0.2, 2], epochs=65)
plot_all_boundaries(X, y, lrs=[0.02,0.2, 2], epochs=95)


In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load dataset
df = pd.read_csv('data.csv', header=None, names=['x1', 'x2', 'y'])
X, y = df[['x1', 'x2']].values, df['y'].values

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def train_logistic(X, y, lr=0.1, epochs=100):
    w = np.random.randn(X.shape[1])
    b = 0.0
    history = [(w.copy(), b)]
    losses = []

    for _ in range(epochs):
        for xi, yi in zip(X, y):
            z = np.dot(w, xi) + b
            y_hat = sigmoid(z)
            error = yi - y_hat
            w += lr * error * xi
            b += lr * error
        history.append((w.copy(), b))

        # Log-loss
        z_all = X.dot(w) + b
        y_hat_all = sigmoid(z_all)
        loss = -np.mean(y * np.log(y_hat_all + 1e-9) + (1 - y) * np.log(1 - y_hat_all + 1e-9))
        losses.append(loss)

    return history, losses

# Subplot for boundaries
def plot_multiple_boundaries(X, y, lrs, epochs):
    fig = make_subplots(rows=1, cols=3, subplot_titles=[f"lr={lr}" for lr in lrs])

    for i, lr in enumerate(lrs, start=1):
        hist, _ = train_logistic(X, y, lr=lr, epochs=epochs)

        for lbl, color in zip([0, 1], ['blue', 'orange']):
            mask = (y == lbl)
            fig.add_trace(go.Scatter(
                x=X[mask, 0], y=X[mask, 1],
                mode='markers', marker=dict(size=6, color=color),
                name=f'Class {lbl}' if i == 1 else None,
                showlegend=(i == 1)
            ), row=1, col=i)

        for j, (w, b) in enumerate(hist):
            x0, x1 = 0.0, 1.0
            if abs(w[1]) > 1e-6:
                y0 = -(w[0] * x0 + b) / w[1]
                y1 = -(w[0] * x1 + b) / w[1]
            else:
                x0 = x1 = -b / w[0]; y0, y1 = 0.0, 1.0

            if j == 0:
                colr, dash, width, name = 'red', 'solid', 3, 'Initial'
            elif j == len(hist) - 1:
                colr, dash, width, name = 'black', 'solid', 3, 'Final'
            else:
                colr, dash, width, name = 'green', 'dash', 2, None

            fig.add_trace(go.Scatter(
                x=[x0, x1], y=[y0, y1],
                mode='lines',
                line=dict(color=colr, dash=dash, width=width),
                name=name if i == 1 and name else None,
                showlegend=(i == 1 and bool(name))
            ), row=1, col=i)

    fig.update_layout(
        title=f"Part 2: Logistic Regression Decision Boundaries (Epochs={epochs})",
        height=500, width=1500
    )

    for i in range(1, 4):
        fig.update_xaxes(range=[0, 1], title_text="x₁", row=1, col=i)
        fig.update_yaxes(range=[0, 1], title_text="x₂", row=1, col=i)

    fig.show()

# Subplot for log loss plots
def plot_multiple_losses(X, y, lrs, epochs):
    fig = make_subplots(rows=1, cols=3, subplot_titles=[f"lr={lr}" for lr in lrs])

    for i, lr in enumerate(lrs, start=1):
        _, losses = train_logistic(X, y, lr=lr, epochs=epochs)
        fig.add_trace(go.Scatter(
            x=list(range(1, len(losses)+1)),
            y=losses,
            mode='lines+markers',
            name=f'lr={lr}',
            marker=dict(color='blue')
        ), row=1, col=i)

    fig.update_layout(
        title_text=f"Part 2: Error Plot (Epochs={epochs})",
        height=500, width=1500
    )

    for i in range(1, 4):
        fig.update_xaxes(title_text="Epoch", row=1, col=i)
        fig.update_yaxes(title_text="Log Loss", row=1, col=i)

    fig.show()

lrs = [0.02, 0.2, 2.0]
plot_multiple_boundaries(X, y, lrs, 35)
plot_multiple_losses(X, y, lrs, 35)
plot_multiple_boundaries(X, y, lrs, 65)
plot_multiple_losses(X, y, lrs, 65)
plot_multiple_boundaries(X, y, lrs, 95)
plot_multiple_losses(X, y, lrs, 95)